In [1]:
import eia_api
import os
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
from ydata_profiling import ProfileReport

In [2]:
log_file = pd.read_csv("data/us48_metadata.csv")

pd.set_option('display.max_rows', len(log_file))
log_file

,index,respondent,respondent_type,time,start,end,start_act,end_act,start_match,end_match,n_obs,na,type,update,success
0,1,US48,Demand,2023-12-31 09:26:33.320023,2015-07-01 05:00:00,2023-12-24 12:00:00,2015-07-01 05:00:00,2023-12-24 12:00:00,True,True,74360.0,0.0,backfill,True,True
1,2,US48,Demand,2023-12-31 12:04:59.398516,2023-12-24 13:00:00,2023-12-31 11:00:00,2023-12-24 13:00:00,2023-12-31 11:00:00,True,True,167.0,0.0,refresh,True,True
2,3,US48,Demand,2023-12-31 12:17:11.597780,2023-12-31 12:00:00,2023-12-31 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,refresh,False,False
3,4,US48,Demand,2023-12-31 13:10:50.045117,2023-12-31 12:00:00,2023-12-31 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,refresh,False,False
4,5,US48,Demand,2023-12-31 15:09:40.191596,2023-12-31 12:00:00,2024-01-01 08:00:00,2023-12-31 12:00:00,2023-12-31 12:00:00,True,False,1.0,0.0,refresh,True,False
5,6,US48,Demand,2023-12-31 17:09:02.679403,2023-12-31 12:00:00,2024-01-01 08:00:00,2023-12-31 12:00:00,2023-12-31 16:00:00,True,False,5.0,0.0,refresh,True,False
6,7,US48,Demand,2023-12-31 18:11:57.917512,2023-12-31 12:00:00,2024-01-01 08:00:00,2023-12-31 12:00:00,2023-12-31 17:00:00,True,False,6.0,0.0,refresh,True,False
7,8,US48,Demand,2023-12-31 19:08:10.276018,2023-12-31 12:00:00,2024-01-01 08:00:00,2023-12-31 12:00:00,2023-12-31 18:00:00,True,False,7.0,0.0,refresh,True,False
8,9,US48,Demand,2023-12-31 20:10:40.633359,2023-12-31 12:00:00,2024-01-01 08:00:00,2023-12-31 12:00:00,2023-12-31 19:00:00,True,False,8.0,0.0,refresh,True,False
9,10,US48,Demand,2023-12-31 21:08:55.196655,2023-12-31 12:00:00,2024-01-01 08:00:00,2023-12-31 12:00:00,2023-12-31 20:00:00,True,False,9.0,0.0,refresh,True,False


In [3]:
log_file = pd.read_csv("data/us48_metadata.csv")

cols_time = log_file.columns[3:8]
log_file[cols_time] = log_file[cols_time].apply(pd.to_datetime)
print(log_file)

     index respondent respondent_type                       time   
0        1       US48          Demand 2023-12-31 09:26:33.320023  \
1        2       US48          Demand 2023-12-31 12:04:59.398516   
2        3       US48          Demand 2023-12-31 12:17:11.597780   
3        4       US48          Demand 2023-12-31 13:10:50.045117   
4        5       US48          Demand 2023-12-31 15:09:40.191596   
5        6       US48          Demand 2023-12-31 17:09:02.679403   
6        7       US48          Demand 2023-12-31 18:11:57.917512   
7        8       US48          Demand 2023-12-31 19:08:10.276018   
8        9       US48          Demand 2023-12-31 20:10:40.633359   
9       10       US48          Demand 2023-12-31 21:08:55.196655   
10      11       US48          Demand 2023-12-31 22:09:29.577372   
11      12       US48          Demand 2023-12-31 23:10:59.121349   
12      13       US48          Demand 2024-01-01 00:51:56.469661   
13      14       US48          Demand 2024-01-01

In [4]:
meta_file = log_file[log_file["success"] == True] 
meta_file = meta_file[meta_file["index"] == meta_file["index"].max()]

end_act = meta_file["end_act"]
print(end_act)
start = end_act + datetime.timedelta(hours = 1)
start = datetime.datetime.strptime(str(start.iloc[0]), "%Y-%m-%d %H:%M:%S")
start

246   2024-01-15 12:00:00
Name: end_act, dtype: datetime64[ns]


datetime.datetime(2024, 1, 15, 13, 0)

In [5]:
api_path = "electricity/rto/region-data/data"
facets = {
    "respondent": "US48",
    "type": "D"
}

offset = 2250

eia_api_key = os.getenv('EIA_API_KEY')

In [6]:
metadata = eia_api.eia_metadata(api_key = eia_api_key, api_path = "electricity/rto/region-data/")

In [9]:
print(metadata.meta["endPeriod"])
end_api = datetime.datetime.strptime(metadata.meta["endPeriod"], "%Y-%m-%dT%H")
print(end_api)
end_fix = end_api -  datetime.timedelta(hours = 12)
print(end_fix)

2024-01-16T08
2024-01-16 08:00:00
2024-01-15 20:00:00


In [10]:
print(start)
print(end_fix)

2024-01-15 13:00:00
2024-01-15 20:00:00


In [13]:
if start <= end_api:
    print("Updates are available")
    df = eia_api.eia_get(api_key = eia_api_key, 
        api_path = api_path, 
        facets = facets, 
        start = start,
        end = end_fix) 
    update_flag = True
    if(len(df.data) > 0):
        start_match_flag = df.data["period"].min() == start
        end_match_flag = df.data["period"].max() == end_fix
        start_act = df.data["period"].min()
        end_act = df.data["period"].max()
        n_obs = len(df.data)
        na = df.data["value"].isna().sum()
        if start_match_flag and end_match_flag and na == 0 and n_obs > 0:
            print("Refresh successed")
            success_flag = True
        else:
            success_flag = False
            print("Refresh failed")
    else:
        print("Refresh failed")
        success_flag = False
        start_match_flag = None
        end_match_flag = None
        start_act = None
        end_act = None
        n_obs = None
        na = None
        
else:
    print("No updates are available...")
    update_flag = False
    success_flag = False
    start_match_flag = None
    end_match_flag = None
    start_act = None
    end_act = None
    n_obs = None
    na = None
    

Updates are available
Refresh successed


In [11]:
df.data

,period,respondent,respondent-name,type,type-name,value,value-units
0,2024-01-04 09:00:00,US48,United States Lower 48,D,Demand,443482,megawatthours


In [13]:
log = {
    "index": log_file["index"].max() + 1,
    "respondent": "US48",
    "respondent_type": "Demand",
    "time": datetime.datetime.now(),
    "start": start,
    "end": end_api,
    "start_act": start_act,
    "end_act": end_act,
    "start_match": start_match_flag, 
    "end_match": end_match_flag, 
    "n_obs": n_obs,
    "na": na,
    "type": "refresh",
    "update": update_flag,
    "success": success_flag

}

log_file_new = pd.DataFrame([log])

log_file_new

,index,respondent,respondent_type,time,start,end,start_act,end_act,start_match,end_match,n_obs,na,type,update,success
0,92,US48,Demand,2024-01-04 10:14:26.338928,2024-01-04 09:00:00,2024-01-04 09:00:00,2024-01-04 09:00:00,2024-01-04 09:00:00,True,True,1,0,refresh,True,True


In [104]:
if (success_flag):
    print("Save the data into CSV file")
    data = pd.read_csv("data/us48.csv")
    new_data = data._append(df.data)
    new_data.to_csv("data/us48.csv", index = False)
    print("Save the metadata into CSV file")
    new_log = log_file._append(log_file_new)
    new_log.to_csv("data/us48_metadata.csv", index = False)
else:
    if not update_flag:
       print("Updates are not available")
    elif update_flag and not success_flag:
        print("Fail to refresh the data, check the log...")
    new_log = log_file._append(log_file_new)
    new_log.to_csv("data/us48_metadata.csv", index = False)

Save the data into CSV file
Save the metadata into CSV file


In [105]:
new_log

,index,respondent,respondent_type,time,start,end,start_act,end_act,start_match,end_match,n_obs,na,type,update,success
0,1,US48,Demand,2023-12-25 00:41:10.371827,2015-07-01 05:00:00,2023-12-15 00:00:00,2015-07-01 05:00:00,2023-12-15 00:00:00,True,True,74132.0,0.0,backfill,True,True
1,2,US48,Demand,2023-12-25 00:41:51.970594,2023-12-15 01:00:00,2023-12-24 12:00:00,2023-12-15 01:00:00,2023-12-24 12:00:00,True,True,228.0,0.0,refresh,True,False
2,3,US48,Demand,2023-12-25 00:42:39.120097,2023-12-24 13:00:00,2023-12-24 12:00:00,NaT,NaT,NaN,NaN,NaN,NaN,refresh,False,False
3,4,US48,Demand,2023-12-25 00:44:56.139032,2023-12-15 01:00:00,2023-12-24 12:00:00,2023-12-15 01:00:00,2023-12-24 12:00:00,True,True,228.0,0.0,refresh,True,False
0,5,US48,Demand,2023-12-25 00:49:42.607598,2023-12-15 01:00:00,2023-12-24 12:00:00,2023-12-15 01:00:00,2023-12-24 12:00:00,True,True,228.0,0.0,refresh,True,True
